In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow

import torch

print('Cuda ststus: %s' % str(torch.cuda.is_available()))
print('Divices available: %d' % torch.cuda.device_count())
print('Device name: %s' % torch.cuda.get_device_name())

Cuda ststus: True
Divices available: 1
Device name: NVIDIA GeForce GTX 1650


### Spark Session

In [2]:
import findspark
findspark.init('C:\spark')

from pyspark.sql import SparkSession

spark = (SparkSession.builder
         .appName('Reviews Session')
         .config('spark.executor.memory', '8g')
         .config('spark.executor.cores', '2')
         .config('spark.driver.memory', '4g')
         .getOrCreate())

spark

### Getting unzipped files from other file path

In [16]:
from file_handling_tools import FileHandling

# This will decompress the multiple zip files.
# Having the input directory where the zip files are, and the output directory where the unzipped files will be stored.

input_directory = r"C:\Users\jdieg\Desktop\henry\proyectos\FINAL\data\google_maps"
output_directory = r"C:\Users\jdieg\Desktop\data_camp\projects\restaurant_reviews\data\google"

file_handling = FileHandling(input_directory, output_directory)

# # Execute the decompression process (Execute only for the first time)
# file_handling.unzip_folders()

### Have a look at Data Files.
On:
* metadata-sitios (json files)
* reviews (json files)

To get fields names, type of data, and so on..

#### metadata-sitios (json file)

In [4]:
# Multi Loading json files for metadata
metadata_df = spark.read.json("data/google/*00[2-3]/*/metadata*/*.json")

In [5]:
metadata_df.count()

3025011

In [26]:
# Print the Schema to have a lot at structure DataFrame
metadata_df.printSchema()

root
 |-- MISC: struct (nullable = true)
 |    |-- Accessibility: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Activities: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Amenities: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Atmosphere: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Crowd: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Dining options: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- From the business: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Getting here: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Health & safety: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Health and safety: array (nullable = true)


In [17]:
from info_spark_dataset import InfoDataSet
info_data = InfoDataSet(metadata_df._jdf, metadata_df.sparkSession)
info_data.display_columns()

Column:
	1.- MISC
	2.- address
	3.- avg_rating
	4.- category
	5.- description
	6.- gmap_id
	7.- hours
	8.- latitude
	9.- longitude
	10.- name
	11.- num_of_reviews
	12.- price
	13.- relative_results
	14.- state
	15.- url


In [7]:
metadata_df.filter('MISC IS NOT NULL').count()

2334177

In [8]:
misc_df = metadata_df.select('MISC')

In [9]:
misc_df.printSchema()

root
 |-- MISC: struct (nullable = true)
 |    |-- Accessibility: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Activities: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Amenities: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Atmosphere: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Crowd: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Dining options: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- From the business: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Getting here: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Health & safety: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- Health and safety: array (nullable = true)


In [10]:
from pyspark.sql.functions import col
from pyspark.sql.types import *
metadata_df.select(col('MISC.Accessibility')).alias('Accessibility')



DataFrame[Accessibility: array<string>]

### Reviews

In [13]:
reviews = spark.read.json('data/google/*/*/reviews-estados/review*/*.json')

In [14]:
reviews.count()

89946359

In [15]:
reviews.printSchema()

root
 |-- gmap_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |-- rating: long (nullable = true)
 |-- resp: struct (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- time: long (nullable = true)
 |-- text: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)

